# 調和励振系の正規形（数値シミュレーション）

吉田勝俊（宇都宮大学）

## 参考情報

- [Pythonで運動方程式を解く(odeint) - Qiita](https://qiita.com/binaryneutronstar/items/ad5efa27fd626826846f)
- [[Python] Numpyの参照、抽出、結合 - Qiita](https://qiita.com/supersaiakujin/items/d63c73bb7b5aac43898a)
- [[Python/matplotlib] FuncAnimationを理解して使う - Qiita](https://qiita.com/osanshouo/items/3c66781f41884694838b)
- [2つの信号間の遅延を推定する - Qiita](https://qiita.com/inoory/items/3ea2d447f6f1e8c40ffa)

In [ ]:
import numpy as np                              #数値計算ライブラリ
from math import sin, cos                       #低機能だが高速な数学関数・定数
from numpy.linalg import norm                   #ベクトルのノルム
from scipy.integrate import odeint              #常微分方程式ライブラリ
import matplotlib.pyplot as plt                 #描画ライブラリ
from matplotlib import rc                       #グラフ調整ライブラリ
from matplotlib.animation import FuncAnimation  #アニメーションライブラリ
#Colab用の設定（グラフィックのインライン表示）
%matplotlib inline

## 調和励振系の運動方程式（原形と正規形）

In [ ]:
def org_funcs():
    '''
    調和励振系（原形 original）の外力と運動方程式
    '''
    def force(t, param):
        '''
        外力 f(t)
        '''
        m, c, k, P, Om = param
        ft = P*sin(Om*t)
        
        return ft 

    def eom(x, t, param):
        '''
        運動方程式
        '''
        m, c, k, P, Om = param
        return np.array([
            x[1],
            -(c/m)*x[1] - (k/m)*x[0] + force(t,param)
        ])

    return (force, eom)

def can_funcs():
    '''
    調和励振系（正規形 cannonical）の外力と運動方程式
    '''
    def force(tau, param):
        '''
        無次元化した外力 h(tau)
        '''
        zeta, A, om = param
        ht = A*sin(om*tau)
        
        return ht 

    def eom(y, tau, param):
        '''
        調和励振系の運動方程式（正規形 cannonical）
        '''
        zeta, A, om = param
        return np.array([
            y[1],
            -2*zeta*y[1] - y[0] + force(tau,param)
        ])

    return (force, eom)

In [ ]:
def Solve(funcs, param, x0, time, new_om=None):
    '''
    調和励振系の数値解を求める関数
    '''
    force, eom = funcs()     #外力と運動方程式の取得
    new_param = param.copy() #元のparamを書き換えぬよう複製
    
    if new_om is not None:
        new_param[-1] = new_om   #最終成分＝外力角振動数
        
    ###運動方程式を数値的に解く
    motion = odeint(
        eom,                #運動方程式を表すユーザ関数
        x0,                 #初期条件
        time,               #時間軸を表す数列
        args=(new_param,)   #運動方程式の係数
    )
    
    ###外力の数列を作る
    forces = np.array([
        force(t, new_param) for t in time
    ])

    return (forces, motion)

## 振動波形の比較（原形と正規形）

In [ ]:
def draw_waves(axs, ts, forces, xs, 
               inlab=r'$f(t)$', outlab=r'$x(t)$',
               ymax1=2, ymax2=2,
               title=None ):
    '''
    外力と変位の振動波形を描画する関数
    '''
    ax1, ax2 = axs #第１軸, 第２軸
    
    ax1.plot(ts, forces,   label=inlab,  color='gray') #入力
    ax2.plot(ts, xs[:, 0], label=outlab, color='red')  #出力

    ax1.set_ylim(-ymax1, ymax1) #縦軸の範囲
    ax2.set_ylim(-ymax2, ymax2)

    ax1.set_ylabel('Input Amplitude',  fontsize=15, color='gray')
    ax2.set_ylabel('Output Amplitude', fontsize=15, color='red')

    ax1.set_xlabel('Time', fontsize=16)

    ax1.legend(fontsize=14, loc='upper left') #凡例の表示
    ax2.legend(fontsize=14, loc='upper right') #凡例の表示

    ax1.grid()   #グリッド線の表示
    
    if title is not None:
        ax1.set_title(title, fontsize=14)
        
        
def output_waves(ts, forces, xs, 
                 inlab=r'$f(t)$', outlab=r'$x(t)$', 
                 ymax1=2, ymax2=2):
    '''
    振動波形の描画を出力する関数
    '''
    fig, ax = plt.subplots(1,1, figsize=(7,3))
    axs = [ax, ax.twinx()]  #第１軸, 第２軸
    
    draw_waves(axs, ts, forces, xs, inlab, outlab, ymax1, ymax2)
    fig.tight_layout()

### 原形の数値解

In [ ]:
org_param = [
    1,    # m: 質量 
    0.5,  # c: 減衰係数
    2,    # k: ばね定数
    1,    # P: 外力振幅
    3,    # Om: 外力角振動数
]

X0 = [
    0,  #初期変位 
    2,  #初速度
]

ts = np.linspace(0,50,500) #時間軸を表す数列（0〜50秒を500等分）
forces, xs = Solve(funcs=org_funcs, param=org_param, x0=X0, time=ts)
output_waves(ts, forces, xs)

### 正規形の数値解

初期条件も，速度については変換が必要です．

- $x(t)=y(\omega_n t)$ が成立するので，$t=0$ を代入すると，$x(0)=y(0)$ になります．
- $x(t)=\omega_n y(\omega_n t)$ が成立するので，$t=0$ を代入すると，$x(0)=\omega_n y(0)$ または $y(0)=x(0)/\omega_n$ になります．

In [ ]:
def convert_can_from_org(org_param, x0, ts):
    '''
    原形のパラメータ，初期値，時間軸を，
    正規形に変換する関数
    '''
    # 原形のパラメータ
    m, c, k, P, Om = org_param
    
    # 対応する正規形のパラメータ
    zeta = c/(2*np.sqrt(m*k))
    om_n = np.sqrt(k/m)
    A = P/k
    om = Om/om_n
    
    can_param = [
        zeta,  # zeta: 減衰比
        A,     # A: 無次元外力振幅
        om,    # om: 無次元外力角振動数
    ]
    
    y0 = x0.copy()     #x0 を変えないため，x0のコピーを作り代入
    y0[1] = x0[1]/om_n #初速度は変換が必要
    
    # 対応する無次元時間
    taus = om_n * ts

    return (can_param, y0, taus, om_n)

In [ ]:
can_param, Y0, taus, om_n = convert_can_from_org(org_param, X0, ts)

forces, ys = Solve(
    funcs=can_funcs, 
    param=can_param, 
    x0=Y0, 
    time=taus, 
)

output_waves(taus, forces, ys, inlab=r'$h(\tau)$', outlab=r'$y(\tau)$')

### 振動波形の比較（変位）

In [ ]:
ss = ts #横軸データは流用

plt.plot(ss, ys[:, 0], 'or', label=r'$y(s)$', ms=2) #正規形の変位
plt.plot(ss, xs[:, 0], '-b', label=r'$x(s)$')       #原形の変位
plt.xlabel(r'$s$', fontsize=16)
plt.ylabel('Displacement', fontsize=16)
plt.legend(fontsize=14)

- $x(t)$ と $y(\tau)$ を単なる１変数関数と見て，同じ横軸 $s$ にプロットすると重なるので，両者の振動波形は確かに相似です．
- 若干ずれて見えますが，これは描画ライブラリの誤差です．

### 振動波形の比較（速度）

In [ ]:
plt.plot(ss, ys[:, 1], 'or', label=r'$\dot y(s)$', ms=2) #正規形の速度
plt.plot(ss, xs[:, 1], '-b', label=r'$\dot x(s)$')       #原形の速度
plt.xlabel(r'$s$', fontsize=16)
plt.ylabel('Velocity', fontsize=16)
plt.legend(fontsize=14)

- 速度は振幅方向にも倍率が掛かっています．
- 変換式 $\dot x = \omega_n \dot y$ で速度を揃えると，両者は重なります．

In [ ]:
xs_from_ys1 = om_n * ys[:, 1]

plt.plot(ss, xs_from_ys1, 'or', label=r'$\omega_n \dot y(s)$', ms=2) #正規形の速度
plt.plot(ss, xs[:, 1], '-b', label=r'$\dot x(s)$')       #原形の速度
plt.xlabel(r'$s$', fontsize=16)
plt.ylabel('Velocity', fontsize=16)
plt.legend(fontsize=14)